In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.aapName('ops').getOrCreate()


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-4068803936451797> in <module>
----> 1 spark = SparkSession.builder.aapName('ops').getOrCreate()

AttributeError: 'Builder' object has no attribute 'aapName'

In [0]:
df= spark.read.options(header=True , inferSchema= True).csv('/FileStore/tables/walmart_stock.csv')

In [0]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

In [0]:
df.columns

Out[6]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [0]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [0]:
df.head(5)

Out[8]: [Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [0]:
result=df.describe()

In [0]:
result.select(result['summary'] ,
             format_number(result['Open'].cast('float') , 2).alias('Open'),
             format_number(result['High'].cast('float') ,2).alias('High'),
             format_number(result['Low'].cast('float') , 2).alias('low'),
             format_number(result['Close'].cast('float'),2).alias('Close'),
             result['Volume'].cast('int').alias('Volume')).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



In [0]:
from pyspark.sql.functions import format_number , col

In [0]:
df1=df.withColumn("HV Retio", col('High')/col('Volume'))

In [0]:
df1.select('HV Retio').show()

+--------------------+
|            HV Retio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [0]:
df.orderBy(df['High'].desc()).head(1)[0][0]

Out[25]: datetime.datetime(2015, 1, 13, 0, 0)

In [0]:
from pyspark.sql.functions import mean

In [0]:
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [0]:
from pyspark.sql.functions import max , min

In [0]:
df.select(max('Volume') , min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [0]:
df.filter('Close < 60').count()

Out[38]: 81

In [0]:
result= df.filter(df['Close'] <60)

In [0]:
from pyspark.sql.functions import count

In [0]:
result.select(count('Close')).show()

+------------+
|count(Close)|
+------------+
|          81|
+------------+



In [0]:
a= df.select(count('Close'))

In [0]:
a.show()

+------------+
|count(Close)|
+------------+
|        1258|
+------------+



In [0]:
(df.filter(df['High'] >80).count() * 1.0/ df.count()) *100

Out[50]: 9.141494435612083

In [0]:
from pyspark.sql.functions import corr

In [0]:
df.select(corr('High' , 'Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [0]:
from pyspark.sql.functions import year

In [0]:
yeardf= df.withColumn("year" , year(df['Date']))

In [0]:
maxdf = yeardf.groupBy('year').max()

In [0]:
maxdf.select('year' ,'max(High)').show()

+----+---------+
|year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [0]:
from pyspark.sql.functions import month 


In [0]:
monthdf= df.withColumn("Month" , month(df['Date']))

In [0]:
from pyspark.sql.functions import avg

In [0]:
avgdf= monthdf.groupBy('Month').avg()

In [0]:
avg1=avgdf.select('Month' , 'avg(Close)')

In [0]:
avg1.orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

